In [1]:
import os
import numpy as np
import h5py

from lib.parameters import Parameters
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

/home/cosmonaut/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##  configuration options
config_file = '/media/nfs/PDP3/model34/parameters.xml'

cfg = Parameters(config_file)

components = ['E_r', 'E_z', 'H_phi', 'rho_beam']

start_frame = cfg.get_frame_number_by_timestamp(cfg.start_time)
end_frame = cfg.get_frame_number_by_timestamp(cfg.end_time)

main_group_name = '/pdp3/result'

In [3]:
# define reader (plain reader used)
reader = PlainReader(cfg.data_path, cfg.system_state_path,
                     [cfg.number_r_grid , cfg.number_z_grid],
                     cfg.frames_per_file, False)
reader.verbose = False

f = h5py.File(os.path.join(cfg.data_path, "data.h5"), 'w')

In [4]:
f.create_group(main_group_name)

<HDF5 group "/pdp3/result" (0 members)>

In [5]:
for compon in components:
    # f.create_group(os.path.join('/pdp3/result', i))
    for frame in range(start_frame, end_frame):
        name = os.path.join('/pdp3/result', str(compon), str(frame))
        ds = reader.get_frame(compon, frame)
        f.create_dataset(name, data=ds)
        print("Imported {}/{}".format(compon, frame))

Imported E_r/0
Imported E_r/1
Imported E_r/2
Imported E_r/3
Imported E_r/4
Imported E_r/5
Imported E_r/6
Imported E_r/7
Imported E_r/8
Imported E_r/9
Imported E_r/10
Imported E_r/11
Imported E_r/12
Imported E_r/13
Imported E_r/14
Imported E_r/15
Imported E_r/16
Imported E_r/17
Imported E_r/18
Imported E_r/19
Imported E_r/20
Imported E_r/21
Imported E_r/22
Imported E_r/23
Imported E_r/24
Imported E_r/25
Imported E_r/26
Imported E_r/27
Imported E_r/28
Imported E_r/29
Imported E_r/30
Imported E_r/31
Imported E_r/32
Imported E_r/33
Imported E_r/34
Imported E_r/35
Imported E_r/36
Imported E_r/37
Imported E_r/38
Imported E_r/39
Imported E_r/40
Imported E_r/41
Imported E_r/42
Imported E_r/43
Imported E_r/44
Imported E_r/45


KeyboardInterrupt: 